<a href="https://colab.research.google.com/github/sanmish4ds/cuckoo/blob/master/chatbot_math.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import nltk

In [7]:
nltk.download('punkt') 
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [23]:
pip install wikipedia

  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp36-none-any.whl size=11686 sha256=48693d99e235eaf70b2d38330ac4086e9f92049bd1e38de4323c23d716f026d0
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia


In [0]:
import random
import string
import re, string, unicodedata
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
import wikipedia as wk
from collections import defaultdict
import warnings
warnings.filterwarnings("ignore")
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel

In [9]:
data=open('/content/real_number.txt',mode='r',errors=None)
raw=data.read().lower()
raw[:1000]

'real numbers 1\n1\n1.1 introduction\nin class ix, you began your exploration of the world of real numbers and encountered\nirrational numbers. we continue our discussion on real numbers in this chapter. we\nbegin with two very important properties of positive integers in sections 1.2 and 1.3,\nnamely the euclid’s division algorithm and the fundamental theorem of arithmetic.\neuclid’s division algorithm, as the name suggests, has to do with divisibility of\nintegers. stated simply, it says any positive integer a can be divided by another positive\ninteger b in such a way that it leaves a remainder r that is smaller than b. many of you\nprobably recognise this as the usual long division process. although this result is quite\neasy to state and understand, it has many applications related to the divisibility properties\nof integers. we touch upon a few of them, and use it mainly to compute the hcf of\ntwo positive integers.\nthe fundamental theorem of arithmetic, on the other hand, has t

In [0]:
sent_tokens = nltk.sent_tokenize(raw)
word_token = nltk.word_tokenize(raw)

'real numbers 1\n1\n11 introduction\nin class ix you began your exploration of the world of real numbers and encountered\nirrational numbers we continue our discussion on real numbers in this chapter we\nbegin with two very important properties of positive integers in sections 12 and 13\nnamely the euclid’s division algorithm and the fundamental theorem of arithmetic\neuclid’s division algorithm as the name suggests has to do with divisibility of\nintegers stated simply it says any positive integer a can be divided by another positive\ninteger b in such a way that it leaves a remainder r that is smaller than b many of you\nprobably recognise this as the usual long division process although this result is quite\neasy to state and understand it has many applications related to the divisibility properties\nof integers we touch upon a few of them and use it mainly to compute the hcf of\ntwo positive integers\nthe fundamental theorem of arithmetic on the other hand has to do something\nwith

In [0]:
def normalize(text):
  remove_punct_dict = dict((ord(punct),None) for punct in string.punctuation)
  word_token = nltk.word_tokenize(raw.translate(remove_punct_dict))
  #remove ascii
  new_words = []
  for word in word_token:
      new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
      new_words.append(new_word)
  #Remove tags
  rmv = []
  for w in new_words:
      text=re.sub("&lt;/?.*?&gt;","&lt;&gt;",w)
      rmv.append(text)
  #pos tagging and lemmatization
  tag_map = defaultdict(lambda : wn.NOUN)
  tag_map['J'] = wn.ADJ
  tag_map['V'] = wn.VERB
  tag_map['R'] = wn.ADV
  lmtzr = WordNetLemmatizer()
  lemma_list = []
  rmv = [i for i in rmv if i]
  for token, tag in nltk.pos_tag(rmv):
      lemma = lmtzr.lemmatize(token, tag_map[tag[0]])
      lemma_list.append(lemma)
  return lemma_list

In [0]:
welcome_input = ("hello", "hi", "greetings", "sup", "what's up","hey",)
welcome_response = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]
def welcome(user_response):
    for word in user_response.split():
        if word.lower() in welcome_input:
            return random.choice(welcome_response)

In [0]:
def generateResponse(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    #vals = cosine_similarity(tfidf[-1], tfidf)
    vals = linear_kernel(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0) or "tell me about" in user_response:
        print("Checking Wikipedia")
        if user_response:
            robo_response = wikipedia_data(user_response)
            return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response
#wikipedia search
def wikipedia_data(input):
    reg_ex = re.search('tell me about (.*)', input)
    try:
        if reg_ex:
            topic = reg_ex.group(1)
            wiki = wk.summary(topic, sentences = 3)
            return wiki
    except Exception as e:
            print("No content has been found")

In [26]:
flag=True
print("My name is Chatterbot and I'm a chatbot. If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response not in ['bye','shutdown','exit', 'quit']):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("Chatterbot : You are welcome..")
        else:
            if(welcome(user_response)!=None):
                print("Chatterbot : "+welcome(user_response))
            else:
                print("Chatterbot : ",end="")
                print(generateResponse(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("Chatterbot : Bye!!! ")

My name is Chatterbot and I'm a chatbot. If you want to exit, type Bye!
hi
Chatterbot : I am glad! You are talking to me
what is real number?
Chatterbot : even though, we have worked only with a few examples, you can see that any
real number which has a decimal expansion that terminates can be expressed as a
rational number whose denominator is a power of 10. also the only prime factors of 10
are 2 and 5. so, cancelling out the common factors between the numerator and the
denominator, we find that this real number is a rational number of the form ,
p
q
 where
the prime factorisation of q is of the form 2n5
m
, and n, m are some non-negative integers.
real number
Chatterbot : even though, we have worked only with a few examples, you can see that any
real number which has a decimal expansion that terminates can be expressed as a
rational number whose denominator is a power of 10. also the only prime factors of 10
are 2 and 5. so, cancelling out the common factors between the numerator an